Source: [VAROS - programação](https://www.youtube.com/watch?v=QSR0RCC3Mes&t=39s)

In [4]:
import math
from pandas_datareader import data as pdr
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
pd.options.mode.chained_assignment = None

AlreadyExistsError: Another metric with the same name already exists.